<a href="https://colab.research.google.com/github/ManelSoengas/NLP_Curs/blob/main/Utilitzant_Transformers_NLP_Tasques.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **NLP Tasques**

---
Es treballarà amb les següents tasques lingüístiques comunes que són essencials per treballar tant amb els models tradicionals de NLP com amb els LLM moderns:

1. Classificació de fitxes
1. Modelatge de llenguatge emmascarat (com BERT)
1. Resum
1. Traducció
1. Formació prèvia al modelatge del llenguatge causal (com ara GPT-2)
1. Resposta a preguntes

Aquestes tasques fonamentals constitueixen la base del funcionament dels grans models lingüístics (LLM) i comprendre'ls és crucial per treballar eficaçment amb els models lingüístics més avançats actuals.


# **Token Classificació**

---

